In [13]:
import pandas as pd

In [14]:
# Reading in all four data sets given to us through the MCM/ICM Zip
summer_only_medal_counts = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\summerOly_medal_counts_hosts.csv') # Rank, NOC, Gold, Silver, Bronze, Total, Year
host_countries           = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\summerOly_hosts.csv') # YEAR, HOST (city, nation)
summer_athlete           = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\summerOly_athletes.csv') # Athlete names, team, NOC, and Medals
summer_programs          = pd.read_csv(r'C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\summerOly_programs.csv', encoding='latin-1') # List of program events

# Merge of `Team_Rankings` and `Host_Countries` Datasets:

In [15]:
host_countries.head()

,Year,Host
0,1896,"Athens, Greece"
1,1900,"Paris, France"
2,1904,"St. Louis, United States"
3,1908,"London, United Kingdom"
4,1912,"Stockholm, Sweden"


In [16]:
summer_only_medal_counts.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Hosting
0,1,United States,11,7,2,20,1896,False
1,2,Greece,10,18,19,47,1896,True
2,3,Germany,6,5,2,13,1896,False
3,4,France,5,4,2,11,1896,False
4,5,Great Britain,2,3,2,7,1896,False


In [17]:
# Define a function to clean up the mistranslated codes
def clean_host(host_df):
  return host_df.replace('\xa0', '')

In [18]:
# Cleaning the locations given in the `hosts` dataset
cleaned_locations = []

for i in host_countries['Host']:
  location = i.split(',')
  cleaned_location = list(map(clean_host, location))
  cleaned_locations.append(cleaned_location)

cleaned_locations[5] = ['Berlin', 'Germany']
cleaned_locations[11] = ['Tokyo', 'Japan']
cleaned_locations[12] = ['London', 'United Kingdom']
cleaned_locations[31] = ['Tokyo', 'Japan']

for i in range(len(cleaned_locations)):
  if cleaned_locations[i][1] == 'United Kingdom':
    cleaned_locations[i] = [cleaned_locations[i][0], 'Great Britain']

cleaned_locations

[['Athens', 'Greece'],
 ['Paris', 'France'],
 ['St. Louis', 'United States'],
 ['London', 'Great Britain'],
 ['Stockholm', 'Sweden'],
 ['Berlin', 'Germany'],
 ['Antwerp', 'Belgium'],
 ['Paris', 'France'],
 ['Amsterdam', 'Netherlands'],
 ['Los Angeles', 'United States'],
 ['Berlin', 'Germany'],
 ['Tokyo', 'Japan'],
 ['London', 'Great Britain'],
 ['London', 'Great Britain'],
 ['Helsinki', 'Finland'],
 ['Melbourne', 'Australia'],
 ['Rome', 'Italy'],
 ['Tokyo', 'Japan'],
 ['Mexico City', 'Mexico'],
 ['Munich', 'West Germany'],
 ['Montreal', 'Canada'],
 ['Moscow', 'Soviet Union'],
 ['Los Angeles', 'United States'],
 ['Seoul', 'South Korea'],
 ['Barcelona', 'Spain'],
 ['Atlanta', 'United States'],
 ['Sydney', 'Australia'],
 ['Athens', 'Greece'],
 ['Beijing', 'China'],
 ['London', 'Great Britain'],
 ['Rio de Janeiro', 'Brazil'],
 ['Tokyo', 'Japan'],
 ['Paris', 'France'],
 ['Los Angeles', 'United States'],
 ['Brisbane', 'Australia']]

In [19]:
#Creating columns that contain solely `City` and `Country`
city = []
country = []

for i in cleaned_locations:
  city.append(i[0])
  country.append(i[1])

host_countries['City'] = city
host_countries['Country'] = country
host_countries

,Year,Host,City,Country
0,1896,"Athens, Greece",Athens,Greece
1,1900,"Paris, France",Paris,France
2,1904,"St. Louis, United States",St. Louis,United States
3,1908,"London, United Kingdom",London,Great Britain
4,1912,"Stockholm, Sweden",Stockholm,Sweden
5,1916,Cancelled (WWI – Berlin had been awarded),Berlin,Germany
6,1920,"Antwerp, Belgium",Antwerp,Belgium
7,1924,"Paris, France",Paris,France
8,1928,"Amsterdam, Netherlands",Amsterdam,Netherlands
9,1932,"Los Angeles, United States",Los Angeles,United States


In [20]:
# PART 1: Determining which observations to add the True/False boolean to in the `Hosting` column.
hosting = []

for country in range(summer_only_medal_counts.shape[0]):
  found = False
  for location in range(host_countries.shape[0]):
    if (int(host_countries.loc[location, 'Year']) == int(summer_only_medal_counts.loc[country, 'Year'])) and (str(host_countries.loc[location, 'Country']) == str(summer_only_medal_counts.loc[country, 'NOC'])):
      hosting.append(True)
      found = True
      break
  if not found:
    hosting.append(False)

len(hosting)

1435

In [21]:
# PART 2: Determining which observations to add the True/False boolean to in the `Hosting` column.
summer_only_medal_counts['Hosting'] = hosting
summer_only_medal_counts[summer_only_medal_counts['Hosting'] == True]
summer_only_medal_counts[summer_only_medal_counts['Hosting'] == True]['NOC'].value_counts()

#Output
summer_only_medal_counts.to_csv('summerOly_medal_counts_hosts.csv', index = False)
summer_only_medal_counts.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Hosting
0,1,United States,11,7,2,20,1896,False
1,2,Greece,10,18,19,47,1896,True
2,3,Germany,6,5,2,13,1896,False
3,4,France,5,4,2,11,1896,False
4,5,Great Britain,2,3,2,7,1896,False


# Standardizing NOC from `Summer_Only_Medal_Host` DF to match `Summer_Athletes`

In [22]:
# Athlete DF from 2000-2020, this way we set aside our testing data of 2024 for final model metrics.
summer_athlete = summer_athlete[summer_athlete['Year'] >= 2000]
summer_athlete = summer_athlete[summer_athlete['Year'] <= 2020]
noc_athlete_list = summer_athlete['NOC'].unique()
print('Total NOC listed in Athlete DF (from 2000-2020): ', len(noc_athlete_list))

# Medal DF from 2000-2020, this way we set aside our testing data of 2024 for final model metrics
summer_only_medal_counts = summer_only_medal_counts[summer_only_medal_counts['Year'] >= 2000]
medal_counts_00_20 = summer_only_medal_counts[summer_only_medal_counts['Year'] <= 2020]
noc_medal_list = medal_counts_00_20['NOC'].unique()
print('Total NOC with Medals (from 2000-2020): ', len(noc_medal_list))

Total NOC listed in Athlete DF (from 2000-2020):  213
Total NOC with Medals (from 2000-2020):  128


In [23]:
# Drop NON-Current Countries or NOC where territories have changed or Balkanized. Additionally, the MCM/ICM Challenge is looking for National Predictions not team predictions, so if a team doesn't correspond with a country it should be dropped.
# Drop rows where medal['NOC'] matches any value in the list
nations_to_drop = ['Australasia', 'Bohemia', 'Mixed Team', 'Refugee Olympic Team', 'Russian Empire']
summer_only_medal_counts = summer_only_medal_counts[~summer_only_medal_counts['NOC'].isin(nations_to_drop)].reset_index(drop=True) #Remember summer only medal counts has years 2000+**

In [24]:
summer_only_medal_counts.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Hosting
0,1,United States,37,24,32,93,2000,False
1,2,Russia,32,28,29,89,2000,False
2,3,China,28,16,14,58,2000,False
3,4,Australia,16,25,17,58,2000,True
4,5,Germany,13,17,26,56,2000,False


In [25]:
summer_athlete.head()

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
7,Jyri Aalto,M,Finland,FIN,2000,Sydney,Badminton,Badminton Men's Singles,No medal
9,Minna Aalto,F,Finland,FIN,2000,Sydney,Sailing,Sailing Women's Windsurfer,No medal
31,Timo Aaltonen,M,Finland,FIN,2000,Sydney,Athletics,Athletics Men's Shot Put,No medal
33,Ragnhild Aamodt,F,Norway,NOR,2008,Beijing,Handball,Handball Women's Handball,Gold


In [26]:
import pycountry

# Function to get country code
def country_code(country_name):
    try:
        country = pycountry.countries.get(name=country_name)
        if country:
            return country.alpha_3
        else:
            return None
    except Exception:
        return None

In [27]:
# REPLACING COUNTRY NAMES WITH CODES (to match the Athlete DF)
# Initialize a list for ISO codes
ISO_Code = []

# Loop through the 'NOC' column in medal DataFrame --- These are additional codes due to a few mis-matches between olympic naming ISO codes and pycountry.
for i in summer_only_medal_counts['NOC']:
    code = country_code(i)
    if code is None:
        # Manual cleaning for countries not found in pycountry
        if i == "Chinese Taipei":
            code = "TPE"
        elif i == "Turkey":
            code = "TUR"
        elif i == "North Korea":
            code = "PRK"
        elif i == "Moldova":
            code = "MDA"
        elif i == "Ivory Coast":
            code = "CIV"
        elif i == "Great Britain":
            code = "GBR"
        # Any other special cases can be added here
    ISO_Code.append(code)

In [28]:
# Add the 'CODE' column to the Medal DataFrame
summer_only_medal_counts['NOC'] = ISO_Code

# Drop rows where 'CODE' is still None
summer_only_medal_counts = summer_only_medal_counts.dropna(subset=['NOC']).reset_index(drop=True)
summer_only_medal_counts.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Hosting
0,1,USA,37,24,32,93,2000,False
1,3,CHN,28,16,14,58,2000,False
2,4,AUS,16,25,17,58,2000,True
3,5,DEU,13,17,26,56,2000,False
4,6,FRA,13,14,11,38,2000,False


In [29]:
#Remove numbers and symbols from the country names
summer_athlete['NOC'] = summer_athlete['NOC'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
summer_only_medal_counts['NOC'] = summer_only_medal_counts['NOC'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

# Sport Name Standardizing 

In [30]:
parsing = [True if ',' in i else False for i in summer_athlete['Sport']]
required_changes = summer_athlete[parsing]

# required_changes.head()

# Cleaning the goofy data values to something usable
for i in required_changes.index:

  if "km" in required_changes.loc[i, 'Sport']:
    summer_athlete.loc[i, 'Sport'] = "Marathon Swimming"
  else:
    summer_athlete.loc[i, 'Sport'] = "Swimming"

  if "Road" in required_changes.loc[i, 'Sport']:
    summer_athlete.loc[i, 'Sport'] = "Cycling Road"
  elif "Cross" in required_changes.loc[i, 'Sport']:
    summer_athlete.loc[i, 'Sport'] = "Cycling Mountain Bike"
  else:
    summer_athlete.loc[i, 'Sport'] = "Cycling Track"

for i in summer_athlete.index:
  if summer_athlete.loc[i, 'Sport'] == "Trampolining":
    summer_athlete.loc[i, 'Sport'] = "Trampoline Gymnastics"

# Output Both Datasets

In [31]:
# Medal and Host DF Output
summer_only_medal_counts.to_csv('Clean_Medal_Host_df.csv', index = False)
summer_only_medal_counts.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Hosting
0,1,USA,37,24,32,93,2000,False
1,3,CHN,28,16,14,58,2000,False
2,4,AUS,16,25,17,58,2000,True
3,5,DEU,13,17,26,56,2000,False
4,6,FRA,13,14,11,38,2000,False


In [32]:
#Summer Athlete Output
summer_athlete.to_csv('Clean_Athlete_df.csv', index=False)
summer_athlete.head()

,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
7,Jyri Aalto,M,Finland,FIN,2000,Sydney,Badminton,Badminton Men's Singles,No medal
9,Minna Aalto,F,Finland,FIN,2000,Sydney,Sailing,Sailing Women's Windsurfer,No medal
31,Timo Aaltonen,M,Finland,FIN,2000,Sydney,Athletics,Athletics Men's Shot Put,No medal
33,Ragnhild Aamodt,F,Norway,NOR,2008,Beijing,Handball,Handball Women's Handball,Gold
